# FAIR-Chem / Open Catalsyt Project Garden

This tutorial demonstrates how Garden can be used ro run pre-trained models from the FAIR-Chem project,
trained on the Open Catalyst Project's OC20 dataset. These models can predict atomic
structure relaxation and other properties relevant to catalysis research.

## Background

The Open Catalyst Project (OC20) is a large-scale dataset of DFT calculations for
catalyst surface reactions. The FAIR-Chem project provides state-of-the-art models
trained on this data to accelerate catalyst discovery.

## Setup

First, let's install the required packages:
- `garden-ai`: Interface with Garden services (https://github.com/Garden-AI/garden)
- `ase`: Atomic Simulation Environment for building structures
- `matplotlib`: Visualization tools

In [ ]:
!pip install garden-ai ase matplotlib

## Import Dependencies

In [ ]:
from ase import Atoms
from ase.build import fcc111, add_adsorbate
import matplotlib.pyplot as plt
from ase.visualize.plot import plot_atoms

## Helper Functions

These functions help us create and visualize atomic structures, and run model predictions.


In [ ]:
def create_test_structure(metal: str = 'Pt', adsorbate: str = 'O') -> Atoms:
    """Create a test surface-adsorbate structure.

    Args:
        metal: Chemical symbol of the metal surface (default: 'Pt')
        adsorbate: Chemical symbol of the adsorbate (default: 'O')

    Returns:
        ASE Atoms object representing the structure
    """
    # Create an FCC(111) surface with 5 layers and 10Å vacuum
    slab = fcc111(metal, size=(2, 2, 5), vacuum=10.0)
    # Add the adsorbate at an FCC hollow site
    add_adsorbate(slab, adsorbate, height=2.5, position='fcc')
    return slab

def visualize_structure(atoms: Atoms, title: str | None = None) -> None:
    """Visualize an atomic structure from two angles.

    Args:
        atoms: ASE Atoms object to visualize
        title: Optional title for the plot
    """
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    # Top view
    plot_atoms(atoms, axs[0])
    axs[0].set_title("Top View")
    # Side view
    plot_atoms(atoms, axs[1], rotation=('-90x'))
    axs[1].set_title("Side View")

    axs[0].set_axis_off()
    axs[1].set_axis_off()

    if title:
        fig.suptitle(title, fontsize=14, y=1.05)
    plt.tight_layout()
    plt.show()

## Create Initial Structure

Let's create a test structure with a Platinum surface and an Oxygen adsorbate.
This represents a typical catalytic system where we want to understand how oxygen
interacts with a platinum surface.

In [ ]:
slab = create_test_structure()
visualize_structure(slab, "Initial Structure: Pt(111) surface with O adsorbate")

## Connect to Garden

Now we'll connect to the Garden service using Globus authentication.
Follow the link provided and paste the access code when prompted.

In [ ]:
from garden_ai import GardenClient
gc = GardenClient()

## Access FAIR-Chem Models via Garden

This [Garden](https://thegardens.ai/#/garden/10.26311%2Ftctw-zq69) contains several state-of-the-art models. We'll access it using its DOI: `10.26311/tctw-zq69`

In [ ]:
garden_doi = "10.26311/tctw-zq69"
fairchem_oc20 = gc.get_garden(garden_doi)

## Run Predictions with EquiformerV2

Let's start with the EquiformerV2_S2EF model. This is a transformer-based model that predicts:
1. The relaxed atomic structure
2. The forces on each atom

First, we'll convert our atomic structure to a format the model can understand:

In [ ]:
# Convert our structure to a dictionary for remote execution
structure_dict = slab.todict()

In [ ]:
# Run the prediction with EquiformerV2
result = fairchem_oc20.EquiformerV2_S2EF.predict(structure_dict)
result

In [ ]:
# Convert the resulting structure back to Atoms
optimized_slab = Atoms.fromdict(result["structure"])

In [ ]:
# Visualize the optimized structure
visualize_structure(
    optimized_slab,
    "EquiformerV2 Optimized Structure"
)

## Try Another Model: DimeNet++

Now let's try the DimeNetPP_S2EF model. This model uses a different architecture
(message-passing neural networks) but predicts the same properties.

We can reuse our same input structure:

In [ ]:
result = fairchem_oc20.DimeNetPP_S2EF.predict(structure_dict)
result

In [ ]:
# Visualize the optimized structure
visualize_structure(
    optimized_slab,
    "DimeNet++ Optimized Structure"
)

## Run Batch Predictions

The models in this Garden also accept a batch of atomic structures.

In [ ]:
structures = [create_test_structure().todict() for _ in range(5)]

# Use eSCN for this run
results = fairchem_oc20.ESCN_S2EF.predict(structures)
results